# Data Exploration and Cleanup

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import seaborn as sns
import os
import datetime
from pprint import pprint
import plotly.express as px


In [61]:
parent_folder = 'historical_data'
csv_list = [f for f in os.listdir(parent_folder) if '.csv' in f]
df_list = []
for csv in csv_list:
    df = pd.read_csv(os.path.join(parent_folder, csv))
    df_list.append(df)
master_df = pd.concat(df_list, ignore_index=True)

In [62]:
master_df.head()

,Date,Country,City,Specie,count,min,max,median,variance
0,6/01/2015,KR,Jeonju,co,124,0.1,12.3,4.5,55.74
1,22/01/2015,KR,Jeonju,co,116,4.5,10.0,6.7,16.09
2,30/03/2015,KR,Jeonju,co,118,1.2,11.2,5.6,35.98
3,27/05/2015,KR,Jeonju,co,93,2.3,5.6,3.4,6.54
4,3/02/2015,KR,Jeonju,co,133,4.5,13.4,7.8,39.24


In [63]:
# Display an overview of the Specie column
master_df["Specie"].unique()

array(['co', 'pm10', 'o3', 'so2', 'no2', 'pm25', 'psi', 'uvi', 'neph',
       'aqi', 'mepaqi', 'pol', 'temperature', 'humidity', 'pressure',
       'wd', 'wind-speed', 'd', 'pm1', 'wind-gust', 'precipitation',
       'dew', 'wind speed', 'wind gust'], dtype=object)

In [64]:
# Display an overview of the number of rows each Specie has in the dataframe
master_df["Specie"].value_counts()

no2              565392
pm25             552899
pm10             547363
o3               541019
so2              469871
co               429370
temperature      338991
humidity         338874
pressure         336973
wind-speed       327923
dew              230768
wind-gust        214730
wd                30465
precipitation     30222
aqi               19417
uvi               14001
wind speed         4517
pol                4243
wind gust          2875
d                  2031
pm1                1560
neph               1537
mepaqi             1298
psi                 363
Name: Specie, dtype: int64

In [65]:
species_to_remove = ["temperature", "humidity", "pressure", "wind-speed", "dew", "wind-gust",
                     "wind speed", "wind gust", "precipitation", "wd", "aqi", "uvi", "pol", "pm1", "mepaqi", "neph"]

short_airdf = master_df[~master_df["Specie"].isin(species_to_remove)].reset_index(drop=True).copy()

In [66]:
short_airdf.head()

,Date,Country,City,Specie,count,min,max,median,variance
0,6/01/2015,KR,Jeonju,co,124,0.1,12.3,4.5,55.74
1,22/01/2015,KR,Jeonju,co,116,4.5,10.0,6.7,16.09
2,30/03/2015,KR,Jeonju,co,118,1.2,11.2,5.6,35.98
3,27/05/2015,KR,Jeonju,co,93,2.3,5.6,3.4,6.54
4,3/02/2015,KR,Jeonju,co,133,4.5,13.4,7.8,39.24


In [67]:
# Overview of the remaining air pollutant species in focus
short_airdf["Specie"].value_counts()

no2     565392
pm25    552899
pm10    547363
o3      541019
so2     469871
co      429370
d         2031
psi        363
Name: Specie, dtype: int64

In [68]:
short_airdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3108308 entries, 0 to 3108307
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Date      object 
 1   Country   object 
 2   City      object 
 3   Specie    object 
 4   count     int64  
 5   min       float64
 6   max       float64
 7   median    float64
 8   variance  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 213.4+ MB


In [69]:
short_airdf["Date"] = pd.to_datetime(short_airdf["Date"], format="%d/%m/%Y")

In [70]:
# Double check the Date data type
short_airdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3108308 entries, 0 to 3108307
Data columns (total 9 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Date      datetime64[ns]
 1   Country   object        
 2   City      object        
 3   Specie    object        
 4   count     int64         
 5   min       float64       
 6   max       float64       
 7   median    float64       
 8   variance  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 213.4+ MB


In [71]:
# Find the earliest date the air quality dataset covers:
short_airdf["Date"].min()

Timestamp('2014-12-29 00:00:00')

In [72]:
# Find the latest date the air quality dataset covers:
short_airdf["Date"].max()

Timestamp('2021-06-24 00:00:00')

In [73]:
# Because we only focus on air data from 2019 to 2020H1, we'll remove the few data points on 2018-12-31 and the first days of July 2020.
clean_airdf = short_airdf[(short_airdf["Date"] >= pd.to_datetime("2019-01-01")) & (short_airdf["Date"] <= pd.to_datetime("2020-07-01"))].copy()
clean_airdf.head()

,Date,Country,City,Specie,count,min,max,median,variance
1647731,2019-01-16,AE,Abu Dhabi,pm10,24,86.0,99.0,97.0,179.40
1647732,2019-01-22,AE,Abu Dhabi,pm10,24,51.0,57.0,55.0,23.75
1647733,2019-01-26,AE,Abu Dhabi,pm10,24,136.0,173.0,160.0,941.96
1647734,2019-01-07,AE,Abu Dhabi,pm10,24,60.0,91.0,72.0,1006.88
1647735,2019-01-10,AE,Abu Dhabi,pm10,24,82.0,93.0,87.0,57.97


In [74]:
# Set the starting date for the air quality analysis
starting_date = clean_airdf["Date"].min()
starting_date

Timestamp('2019-01-01 00:00:00')

In [75]:
# Set the cutoff date for the air quality analysis
end_date = clean_airdf["Date"].max()
end_date

Timestamp('2020-01-05 00:00:00')

In [76]:
# Display an overview of the Country column
clean_airdf["Country"].unique()

array(['AE', 'CZ', 'SG', 'AT', 'BE', 'KR', 'MN', 'NO', 'NP', 'PE', 'BA',
       'BR', 'IL', 'IN', 'IR', 'NZ', 'UA', 'US', 'FR', 'KW', 'RE', 'TR',
       'XK', 'UZ', 'FI', 'JP', 'KZ', 'RO', 'RU', 'SK', 'ES', 'HK', 'HR',
       'TH', 'LK', 'AR', 'CA', 'CH', 'CL', 'ET', 'KG', 'CW', 'DE', 'MX',
       'SA', 'UG', 'BD', 'HU', 'IS', 'PH', 'ID', 'MO', 'SE', 'SV', 'VN',
       'AU', 'BH', 'GB', 'IE', 'LA', 'LT', 'PR', 'PT', 'TW', 'CN', 'PL',
       'BG', 'CY', 'IQ', 'MK', 'BO', 'DK', 'EC', 'ZA', 'CO', 'EE', 'IT',
       'JO', 'NL', 'RS', 'PK', 'TJ', 'MM', 'DZ', 'TM', 'AF', 'GR', 'ML',
       'GE', 'GT', 'CR'], dtype=object)

In [77]:
clean_airdf["Country"].nunique()

91

In [78]:
# Display an overview of the City column
clean_airdf["City"].unique()

array(['Abu Dhabi', 'Dubai', 'Prague', 'Ostrava', 'Brno', 'Pilsen',
       'Olomouc', 'Singapore', 'Salzburg', 'Linz', 'Innsbruck', 'Vienna',
       'Graz', 'Liège', 'Brussels', 'Charleroi', 'Gent', 'Namur',
       'Antwerpen', 'Jeonju', 'Seoul', 'Busan', 'Sejong', 'Ulsan',
       'Yeosu', 'Gwangju', 'Seongnam-si', 'Daejeon', 'Pohang', 'Changwon',
       'Suwon', 'Cheongju-si', 'Chuncheon', 'Daegu', 'Jeju City',
       'Incheon', 'Suncheon', 'Mokpo', 'Andong', 'Ulan Bator',
       'Trondheim', 'Oslo', 'Stavanger', 'Bergen', 'Kathmandu', 'Pokhara',
       'Lima', 'Sarajevo', 'Zenica', 'Tuzla', 'São José dos Campos',
       'São Paulo', 'Tel Aviv', 'Haifa', 'Jerusalem', 'Ashdod', 'Netanya',
       'Ashkelon', 'Petaẖ Tiqwa', 'New Delhi', 'Hyderabad', 'Delhi',
       'Chandigarh', 'Bhopal', 'Lucknow', 'Ghāziābād', 'Hāpur',
       'Gandhinagar', 'Chennai', 'Nashik', 'Visakhapatnam', 'Mumbai',
       'Jaipur', 'Patna', 'Muzaffarnagar', 'Thiruvananthapuram',
       'Bengaluru', 'Kolkata', 'Is

In [79]:
clean_airdf["City"].nunique()

597

In [80]:
clean_airdf.loc[clean_airdf["Country"] == "AU", "City"].value_counts()

Sydney        2242
Brisbane      2220
Wollongong    2181
Melbourne     2172
Newcastle     2117
Darwin        2110
Adelaide      2099
Perth         2085
Hobart         752
Launceston     752
Canberra       170
Name: City, dtype: int64